### Dependencies

In [15]:
import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.python.client import device_lib 

#import pandas as pd

# import train_test_split

In [2]:
training_csv = 'iris_training.csv'
testing_csv = 'iris_test.csv'
full_set_csv = 'iris_data.csv'

### Environment info

NVIDIA® GPU drivers —CUDA 9.0 requires 384.x or higher. \
CUDA® Toolkit —TensorFlow supports CUDA 9.0. \
CUPTI ships with the CUDA Toolkit. ]
cuDNN SDK (>= 7.2) Note: Make sure your GPU has compute compatibility >3.0 \
(Optional) NCCL 2.2 for multiple GPU support. \
(Optional) TensorRT 4.0 to improve latency and throughput for inference on some models.

In [14]:
# Outputs should all end up in the log
# (maybe also save/compress models used for manual/later inspection?)

#import logging
#
#logname = '../output/tf-cuda.log'

In [3]:
! lspci | grep -i nvidia

01:00.0 VGA compatible controller: NVIDIA Corporation GM204 [GeForce GTX 980] (rev a1)
01:00.1 Audio device: NVIDIA Corporation GM204 High Definition Audio Controller (rev a1)


In [4]:
! lspci | grep ' VGA ' | cut -d" " -f 1 | xargs -i lspci -v -s {}

01:00.0 VGA compatible controller: NVIDIA Corporation GM204 [GeForce GTX 980] (rev a1) (prog-if 00 [VGA controller])
	Subsystem: eVga.com. Corp. GM204 [GeForce GTX 980]
	Physical Slot: 3
	Flags: bus master, fast devsel, latency 0, IRQ 44, NUMA node 0
	Memory at c5000000 (32-bit, non-prefetchable) [size=16M]
	Memory at b0000000 (64-bit, prefetchable) [size=256M]
	Memory at c0000000 (64-bit, prefetchable) [size=32M]
	I/O ports at 6000 [size=128]
	Expansion ROM at 000c0000 [disabled] [size=128K]
	Capabilities: <access denied>
	Kernel driver in use: nouveau
	Kernel modules: nvidiafb, nouveau, nvidia_drm, nvidia

07:00.0 VGA compatible controller: ASPEED Technology, Inc. ASPEED Graphics Family (rev 30) (prog-if 00 [VGA controller])
	Subsystem: ASUSTeK Computer Inc. ASPEED Graphics Family
	Flags: medium devsel, IRQ 16, NUMA node 0
	Memory at c2000000 (32-bit, non-prefetchable) [size=32M]
	Memory at c4000000 (32-bit, non-prefetchable) [size=128K]
	I/O ports at 3000 [size=128]
	Capabilities: <

In [22]:
! grep LoadModule /var/log/Xorg.0.log

[    93.782] (II) LoadModule: "glx"
[    93.861] (II) LoadModule: "nvidia"
[    93.873] (II) LoadModule: "fb"
[    93.874] (II) LoadModule: "wfb"
[    93.875] (II) LoadModule: "ramdac"
[    94.875] (II) LoadModule: "dri2"
[    94.909] (II) LoadModule: "evdev"


In [21]:
! grep Driver /etc/X11/xorg.conf

[    93.782] (II) LoadModule: "glx"
[    93.861] (II) LoadModule: "nvidia"
[    93.873] (II) LoadModule: "fb"
[    93.874] (II) LoadModule: "wfb"
[    93.875] (II) LoadModule: "ramdac"
[    94.875] (II) LoadModule: "dri2"
[    94.909] (II) LoadModule: "evdev"
grep: /etc/X11/xorg.conf: No such file or directory


In [5]:
! conda list | fgrep -i -e tensor

tensorboard               2.8.0                    pypi_0    pypi
tensorboard-data-server   0.6.1                    pypi_0    pypi
tensorboard-plugin-wit    1.8.0                    pypi_0    pypi
tensorflow                2.8.0                    pypi_0    pypi
tensorflow-base           2.2.0           gpu_py37h8a81be8_0  
tensorflow-estimator      2.7.0                    pypi_0    pypi
tensorflow-gpu            2.2.0                h0d30ee6_0  
tensorflow-io-gcs-filesystem 0.24.0                   pypi_0    pypi


In [6]:
! conda list | fgrep -i -e cud

cudatoolkit               10.1.243             h6bb024c_0  
cudnn                     7.6.5                cuda10.1_0  


In [7]:
! nvidia-smi

/bin/sh: 1: nvidia-smi: not found


In [8]:
! which nvcc
! ldconfig -p | grep cuda

/usr/bin/nvcc
	libicudata.so.66 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libicudata.so.66
	libicudata.so (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libicudata.so
	libcudart.so.10.1 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libcudart.so.10.1
	libcudart.so (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libcudart.so
	libcuda.so.1 (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libcuda.so.1
	libcuda.so (libc6,x86-64) => /usr/lib/x86_64-linux-gnu/libcuda.so


#### Checking what tensorflow sees

In [26]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [27]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10642385634769806508
xla_global_id: -1
]


In [20]:
import sys
import numpy as np
import tensorflow as tf
from datetime import datetime

device_name = sys.argv#[1]  # Choose device from cmd line. Options: gpu or cpu
device_name
#shape = (int(sys.argv[2]), int(sys.argv[2]))
#if device_name == "gpu":
#    device_name = "/gpu:0"
#else:
#    device_name = "/cpu:0"
#
#with tf.device(device_name):
#    random_matrix = tf.random_uniform(shape=shape, minval=0, maxval=1)
#    dot_operation = tf.matmul(random_matrix, tf.transpose(random_matrix))
#    sum_operation = tf.reduce_sum(dot_operation)
#
#
#startTime = datetime.now()
#with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
#        result = session.run(sum_operation)
#        print(result)
#
## It can be hard to see the results on the terminal with lots of output -- add some newlines to improve readability.
#print("\n" * 5)
#print("Shape:", shape, "Device:", device_name)
#print("Time taken:", datetime.now() - startTime)
#
#print("\n" * 5)

['/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py',
 '-f',
 '/home/bailey/.local/share/jupyter/runtime/kernel-89840218-868b-46a8-9626-255287910b02.json']

### Demo with iris data
Code sampled from: https://rubikscode.net/2021/08/03/introduction-to-tensorflow-with-python-example/ \
Dataset from: https://archive.ics.uci.edu/ml/datasets/iris

#### CPU

In [ ]:
COLUMN_NAMES = [
        'SepalLength', 
        'SepalWidth',
        'PetalLength', 
        'PetalWidth', 
        'Species'
        ]

columns_feat = [
    tf.feature_column.numeric_column(key='SepalLength'),
    tf.feature_column.numeric_column(key='SepalWidth'),
    tf.feature_column.numeric_column(key='PetalLength'),
    tf.feature_column.numeric_column(key='PetalWidth')
]

In [ ]:
train_dataset = pd.read_csv( training_csv , names=COLUMN_NAMES, header=0)

test_dataset = pd.read_csv(testing_csv, names=COLUMN_NAMES, header=0)

In [ ]:
train_x = train_dataset.iloc[:, 0:4]
train_y = train_dataset.iloc[:, 4]

test_x = test_dataset.iloc[:, 0:4]
test_y = test_dataset.iloc[:, 4]

In [ ]:
def train_function( inputs, outputs, batch_size ):
    dataset = tf.data.Dataset.from_tensor_slices(( dict(inputs), outputs ))
    dataset = dataset.shuffle( 1000 ).repeat().batch( batch_size )
    
    return dataset.make_one_shot_iterator().get_next()

def evaluation_function( attributes, classes, batch_size ):
    attributes = dict( attributes )
    
    if classes is None:
        inputs = attributes
    else:
        inputs = (attributes, classes)
        
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    assert batch_size is not None, "batch_size must not be None"
    
    dataset = dataset.batch(batch_size)
    
    return dataset.make_one_shot_iterator().get_next()

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = columns_feat,
    hidden_units = [10, 10],
    n_classes = 3
)

In [ ]:
classifier.train(
    input_fn = lambda : train_function( train_x, train_y, 100 ),
    steps = 1000
)

In [ ]:
eval_result = classifier.evaluate(
    input_fn = lambda : evaluation_function( test_x, test_y, 100 )
)

#### GPU

In [ ]:
data = pd.read_csv( full_set_csv, names = COLUMN_NAMES, header = 0 )
data.head()

In [ ]:
data.dtypes

In [ ]:
data['Species'] = data['Species'].astype("category")
data.dtypes

In [ ]:
print(data.isnull().sum())

In [ ]:
corrMatt = data[ COLUMN_NAMES ].corr()

mask = np.array( corrMatt )
mask[ np.tril_indices_from( mask ) ] = False

fig,ax = plt.subplots()
fig.set_size_inches(20,10)

sn.heatmap( corrMatt, mask = mask, vmax = .8, square = True, annot = True )

In [ ]:
output_data = data["Species"]
input_data = data.drop( "Species", axis = 1 )

X_train, X_test, y_train, y_test = train_test_split( input_data, output_data, test_size=0.3, random_state=72 )

In [ ]:
class IrisClassifier( Model ):
    def __init__( self ):
        super( IrisClassifier, self ).__init__()
        
        self.layer1 = Dense(10, activation='relu')
        self.layer2 = Dense(10, activation='relu')
        self.outputLayer = Dense(3, activation='softmax')

    def call( self, x ):
        x = self.layer1( x )
        x = self.layer2( x )
        
        return self.outputLayer( x )

In [ ]:
model = IrisClassifier()

model.compile( optimizer = tf.keras.optimizers.Adam(),
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']
             )

In [ ]:
model.fit( X_train, y_train, epochs = 300, batch_size = 10 )

In [ ]:
scores = model.evaluate(X_test, y_test)

print("\nAccuracy: %.2f%%" % (scores[1]*100))

In [ ]:
prediction = model.predict( X_test )
prediction1 = pd.DataFrame({'IRIS1':prediction[:,0],'IRIS2':prediction[:,1], 'IRIS3':prediction[:,2]})
prediction1.round( decimals = 4 ).head()